In [82]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

# 8 Functions and Tables

### 8.0.1 Defining a Function

- 给出一个函数的定义

In [ ]:
def double(x):              # Signature
    """ Double x """        # Doucumentation ("docstring"), Body is as below
    return 2 * x            

<!-- ![functionDef](./img/function_definition.jpg) -->

- 几个部分
    - Signature
        - 指定了调用方式, 例如 `double(3)`
    - Doucumentation
        - Text that describe what the function does
        - 通常是 triple-quoted (可以 span 到多行)
    - Body
    - Indentation
        - Each line of code in the body is indented
        - 缩进通常是 2 or 4 spaces (spaces 数量一致即可)
        - The function's body ends at any unindented line
- 上例中 `x` has **local scope**: We can’t refer to x outside the body of `double`

In [ ]:
double(-.6 / 4)              # call expressions

In [ ]:
# 惊人的是还可以 double 一个数组
from datascience import *
double(make_array(3, 4, 5))

In [ ]:
# A function with more than one argument
def percent(x, total):
    """Convert x to a percentage of total.
    
    More precisely, this function divides x by total,
    multiplies the result by 100, and rounds the result
    to two decimal places.
    
    >>> percent(4, 16)
    25.0
    >>> percent(1, 6)
    16.67
    """
    return round((x/total)*100, 2)

help(percent)  # print 出 docstring 中的内容
percent(33, 200)

In [ ]:
import numpy as np
def percents(counts):
    """Convert the values in array_x to percents out of the total of array_x."""
    total = counts.sum()
    return np.round((counts/total)*100, 2)

percents(make_array(7, 4, 10))

In [ ]:
def biggest_difference(array_x):
    """Find the biggest difference in absolute value between two adjacent elements of array_x."""
    diffs = np.diff(array_x)
    absolute_diffs = abs(diffs)
    return max(absolute_diffs)

some_numbers = make_array(2, 4, 5, 6, 4, -1, 1)
big_diff = biggest_difference(some_numbers)
print("The biggset difference is ", big_diff)

- 润上个 cell, 执行的流程如下
    ![function_execution](img/function_execution.jpg)
    
### 8.0.2 Multiple Arguments

- 有参数默认值机制

In [ ]:
def percents(counts, decimal_places):
    """Convert the values in array_x to percents out of the total of array_x."""
    total = counts.sum()
    return np.round((counts/total)*100, decimal_places)

parts = make_array(2, 1, 4)
print("Rounded to 1 decimal place: ", percents(parts, 1))
print("Rounded to 2 decimal places:", percents(parts, 2))
print("Rounded to 3 decimal places:", percents(parts, 3))

In [ ]:
# 以下指定第二个参数默认值为 2
def percents(counts, decimal_places=2):
    """Convert the values in array_x to percents out of the total of array_x."""
    total = counts.sum()
    return np.round((counts/total)*100, decimal_places)

parts = make_array(2, 1, 4)
print("Rounded to 1 decimal place:", percents(parts, 1))
print("Rounded to the default number of decimal places:", percents(parts))

---

## 8.1 Applying a Function to a Column

In [ ]:
def cut_off_at_100(x):
    """The Smaller of x and 100"""
    return min(x, 100)

ages = Table().with_columns(
    'Person', make_array('A', 'B', 'C', 'D', 'E', 'F'),
    'Age', make_array(17, 117, 52, 100, 6, 101)
)
ages

### 8.1.1 apply

- Table method `apply()` 将所定义的函数应用到 table 中的某一列

In [ ]:
ages.apply(cut_off_at_100, 'Age')

In [ ]:
ages.with_column(
    'Cut Off Age', ages.apply(cut_off_at_100, 'Age')
)

### 8.1.2 Functions as Values

- py 中，函数 `cut_off_by_100` 也属于一个 value.
    - 并且像其他 value 一样可以赋值

In [ ]:
cut_off_at_100

In [ ]:
cut_off = cut_off_at_100
cut_off

### 8.1.3 Example: Prediction

In [ ]:
family_heights = Table.read_table('family_heights.csv').drop(3)
family_heights

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

parent_average = (family_heights.column('father') + family_heights.column('mother')) / 2
heights = Table().with_columns(
    'Parent Average', parent_average,
    'Child', family_heights.column('childHeight')
)
heights.show(9)
heights.scatter('Parent Average')

In [ ]:
heights.scatter('Parent Average')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2)
plots.scatter(68, 67.62, color='gold', s=40); # 加分号则不打印 <matplotlib.collections.PathCollection at 0x7f838e8fe3d0>

In [ ]:
close_to_68 = heights.where('Parent Average', are.between(67.5, 68.5))
close_to_68.show(9)
np.average(close_to_68.column('Child')) # predicted height

In [ ]:
# 可以给出一个预测升高的函数
def predict_child(p_avg):
    """Predict the height of a child whose parents have a parent average height of p_avg.
    
    The prediction is the average height of the children whose parent average height is
    in the range p_avg plus or minus 0.5.
    """
    close_points = heights.where('Parent Average', are.between(p_avg-.5, p_avg+.5))   # 可以直接访问函数外的 value！
    return np.average(close_points.column('Child'))

predict_child(68)

In [ ]:
# 将函数 predict_child() 用于表中
heights_with_predictions = heights.with_column(
    'Prediction', heights.apply(predict_child, 'Parent Average')
)
heights_with_predictions
# 润这个 cell 真慢..

In [ ]:
# 考虑来 draw overlaid scatter plots
heights_with_predictions.scatter('Parent Average')

- 称上图中 graph of gold dots 为 **graph of averages**
- 可看出结果大致为一条直线，称其为 **regression line**


---

## 8.2 Classifying by One Variable

### 8.2.1 Counting the Number in Each Category

- Table method `group()` 用于计算每个类别的行数（ver. 单个参数）
    - The result contains one row per unique value in the grouped column.
    - 生成的表的第二列默认 label 为 'count'

In [83]:
cones = Table().with_columns(
    'Flavor', make_array('strawberry', 'chocolate', 'chocolate', 'strawberry', 'chocolate'),
    'Price', make_array(3.55, 4.75, 6.55, 5.25, 5.25)
)
cones.show()
cones.group('Flavor').show()

Flavor,Price
strawberry,3.55
chocolate,4.75
chocolate,6.55
strawberry,5.25
chocolate,5.25


Flavor,count
chocolate,3
strawberry,2


### 8.2.2 Finding a Characteristic of Each Category

- `group()`（ver. 两个参数）
    - 第二个参数为一个函数名，用于 aggregate values in other columns for all of those rows
    - 生成的新表的列数与原表相同

In [84]:
cones.group('Flavor', sum).show()
cones.group('Flavor', max).show()
cones.group('Flavor', len).show()   # 与一个参数的版本相同 ('Price len' --> 'count')

Flavor,Price sum
chocolate,16.55
strawberry,8.8


Flavor,Price max
chocolate,6.55
strawberry,5.25


Flavor,Price len
chocolate,3
strawberry,2


In [85]:
# 实现一下 group() 所干的事情
cones_choc = cones.where('Flavor', are.equal_to('chocolate')).column('Price')
cones_strawb = cones.where('Flavor', are.equal_to('strawberry')).column('Price')
grouped_cones = Table().with_columns(
    'Flavor', make_array('chocolate', 'strawberry'),
    'Array of All the Prices', make_array(cones_choc, cones_strawb)
)
price_totals = grouped_cones.with_column(
    'Sum of the Array', make_array(sum(cones_choc), sum(cones_strawb))
)
price_totals

Flavor,Array of All the Prices,Sum of the Array
chocolate,[4.75 6.55 5.25],16.55
strawberry,[3.55 5.25],8.8


### 8.2.3 Example: NBA Saleries

In [ ]:
nba1 = Table.read_table('nba_salaries.csv')
nba = nba1.relabeled("'15-'16 SALARY", 'SALARY')
nba.show(5)

# 1. How much did each team pay for its player's salaries?
teams_and_money = nba.select('TEAM', 'SALARY')
teams_and_money.group('TEAM', sum).show(5)

# 2. How many NBA players were there in each of the five position?
nba.group('POSITION').show()

# 3. What was the average salary of the players at each of the five positions?
position_and_money = nba.select('POSITION', 'SALARY')
position_and_money.group('POSITION', np.mean).show()

# 在 3 中，如果直接对 nba group(), 则非数字列为空白
nba.group('POSITION', np.mean)

## 8.3 Cross-Classifying by More than One Variable

### 8.3.1 Two Variables: Counting the Number in Each Paired Category

- 可以给 `group()` 的第一个参数传入一个 string (label) array / list

In [ ]:
more_cones = Table().with_columns(
    'Flavor', make_array('strawberry', 'chocolate', 'chocolate', 'strawberry', 'chocolate', 'bubblegum'),
    'Color', make_array('pink', 'light brown', 'dark brown', 'pink', 'dark brown', 'pink'),
    'Price', make_array(3.55, 4.75, 5.25, 5.25, 5.25, 4.75)
)
more_cones.show()
more_cones.group(['Flavor', 'Color']).show()      # 这里使用了 [], 使用 make_array() 也可以

### 8.3.2 Two Variables: Finding a Characteristic of Each Paired Category



In [ ]:
more_cones.group(['Flavor', 'Color'], sum)

### 8.3.3 Pivot Tables: Rearranging the Output of group

> 数据透视表：重新排列组合的输出

- 交叉分类大多仅涉及两个分类变量，此时可在 **pivot table** 显示分类结果
- Pivot tables a.k.a **contingency table** (列联表)
- 使用 Table method `pivot()` 来作 pivot tables
    - 两个参数
        - 第一个参数为列 label
        - 第二个参数 is the label of a column used for the rows
    - 第三个可选参数 `values` indicates a column of values that will replace the counts in each cell of the grid
    - 第四个可选参数 `collect` 指定如何将 valuse 聚合显示在单元格中。(传入函数名)

In [ ]:
more_cones.pivot('Flavor', 'Color').show()
more_cones.pivot('Flavor', 'Color', values='Price', collect=sum)

### 8.3.4 Example: Education and Income of CA Adults

In [ ]:
full_table = Table.read_table('educ_inc.csv')
ca_2014 = full_table.where('Year', are.equal_to('1/1/14 0:00')).where('Age', are.not_equal_to('00 to 17'))
ca_2014.show(9)

In [ ]:
# Foucus on just one pair: educational attainment and perosonal income.
educ_inc = ca_2014.select('Educational Attainment', 'Personal Income', 'Population Count')
educ_inc.show(9)

In [ ]:
# 先看 education
education = educ_inc.select('Educational Attainment', 'Population Count')
educ_totals = education.group('Educational Attainment', sum)
educ_totals.show()

In [ ]:
# 不妨列出上表的百分比
def percents(arr):
    return np.round(arr / sum(arr) * 100, 2)

educ_distribution = educ_totals.with_column(
    'Population Percent', percents(educ_totals.column(1))
)
educ_distribution.show()

In [ ]:
# 看教育程度和个人收入交叉分类的列链表
totals = educ_inc.pivot('Educational Attainment', 'Personal Income', values='Population Count', collect=sum)
totals.show()

In [ ]:
# 考虑将上表表示为百分比
distributions = totals.select(0).with_columns(
    totals.labels[1], percents(totals.column(1)),
    totals.labels[2], percents(totals.column(2)),
    totals.labels[3], percents(totals.column(3)),
    totals.labels[4], percents(totals.column(4))   
)
distributions.show()

In [ ]:
# 看 overlaid bar chart
distributions.select(0, 1, 4).barh(0)

---

## 8.4 Joining Tables by Columns

- Table method `join()`，连接两表
- 示例：使用 table2 的信息来扩充 table1
    ```python
    table1.join(table1_column_for_joining, table2, table2_column_for_joining)
    ```


In [ ]:
cones = Table().with_columns(
    'Flavor', make_array('strawberry', 'vanilla', 'chocolate', 'strawberry', 'chocolate'),
    'Price', make_array(3.55, 4.75, 6.55, 5.25, 5.75)
)
cones.show()
ratings = Table().with_columns(
    'Kind', make_array('strawberry', 'chocolate', 'vanilla'),
    'Stars', make_array(2.5, 3.5, 4)
)
ratings.show()

In [ ]:
rated = cones.join('Flavor', ratings, 'Kind')
rated.show()

# 调换两个表的位置，只造成行列的顺序不一样，无根本性的区别
ratings.join('Kind', cones, 'Flavor').show()

- 注意使用 `join()` 的结果只会包含的两表共有的 item

In [ ]:
reviews = Table().with_columns(
    'Flavor', make_array('vanilla', 'chocolate', 'vanilla', 'chocolate'),
    'Stars', make_array(5, 3, 5, 4)
)
reviews.show()
average_reviews = reviews.group('Flavor', np.mean)
average_reviews.show()
cones.show()

# 将 cones 和 average_reviews 合并
cones.join('Flavor', average_reviews, 'Flavor')  # 只会包含共有的 chocolate and vanilla

## 8.5 Bike Sharing in the Bay Area

In [ ]:
trips = Table.read_table('trip.csv')
trips

In [ ]:
# 只关注 During 1800s 以下的
commute = trips.where('Duration', are.below(1800))
commute.hist('Duration', unit='Second')

In [ ]:
# 可以观察更多细节
commute.hist('Duration', bins=60, unit='Second')

### 8.5.1 Exploring the Data with group() and pivot()

In [ ]:
# 观察使用率最高的 Start station
starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
# group() 也可以同时按起点站和终点站分类
commute.group(['Start Station', 'End Station'])

In [ ]:
# 改用透视图
commute.pivot('Start Station', 'End Station')

In [ ]:
# 还可找到起点和终点之间最短的 duration
commute.pivot('Start Station', 'End Station', values='Duration', collect=min)

### 8.5.2 Drawing Maps

- `Marker.map_table()`
- 还可用彩色圈圈代替地图上的点 `Circle.map_tables()`

In [ ]:
stations = Table.read_table('station.csv')  # 每个 bike station 地理信息
stations

In [ ]:
Marker.map_table(stations.select('lat', 'long', 'name').relabel('name', 'labels'))

In [ ]:
# 还可用彩色圈圈代替地图上的圈
sf = stations.where('landmark', are.equal_to('San Francisco'))
sf_map_data = sf.select('lat', 'long', 'name').relabel('name', 'labels')
Circle.map_table(sf_map_data, color='green')

### 8.5.3 More Infomation Maps: An Application of join()

In [ ]:
# 给每个城市配色, 方法是将同一城市 group 起来，让后添加 color label, 再 join 到原表 (stations)
cities = stations.group('landmark').relabeled('landmark', 'city')
cities.show()
colors = cities.with_column(
    'color', make_array('blue', 'red', 'green', 'orange', 'purple')
)
colors.show()

In [ ]:
joined = stations.join('landmark', colors, 'city')
colored = joined.select('lat', 'long', 'name', 'color').relabel('name', 'labels')
Marker.map_table(colored)

In [ ]:
# see where most of the bike rentals originate
starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
# 将 count 添加到表 station
station_starts = stations.join('name', starts, 'Start Station')
station_starts

In [ ]:
# 考虑用圈来标记每个站点，圈的大小设置为租赁数 count * 0.3
starts_map_data = station_starts.select('lat', 'long', 'name').with_columns(
    'colors', 'blue',
    'areas', station_starts.column('count') * .2
)
starts_map_data.show(3)
Circle.map_table(starts_map_data.relabel('name', 'labels'))